In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'd:\\MLOps-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_callback_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callback_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callback_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2024-01-04 13:04:49,795: WARNING: module_wrapper: From d:\MLOps-Project\MlOps\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [8]:
class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:

try:
    config = ConfigurationManager()
    callbacks_config = config.get_callback_config()
    callbacks = Callbacks(config=callbacks_config)
    callback_list = callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-01-04 13:04:52,186: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-04 13:04:52,190: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-04 13:04:52,191: INFO: common: created directory at: artifacts]
[2024-01-04 13:04:52,193: INFO: common: created directory at: artifacts\callbacks\checkpoint_dir]
[2024-01-04 13:04:52,194: INFO: common: created directory at: artifacts\callbacks\tensorboard_log_dir]
